In [92]:
import jieba.posseg as pseg
import codecs
import numpy as np
import math

In [93]:
#构建停用词表
def stopwords_list(filepath):
    stop_words_file = filepath
    stopwords = codecs.open(stop_words_file,'r',encoding='utf8').readlines()
    stopwords_list = [words.strip() for words in stopwords]
    stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
    return stopwords_list,stop_flag

#对一篇文章，或者句子分词、去停用词
def tokenization(filename):
    res = []
    with open(filename,'r') as f:
        text = f.read()
        words = pseg.cut(text)
    stopwordslist,stop_flag = stopwords_list('/Users/taoxudong/huli/test/stop_words.txt')
    #分词特性？中文对应着stop_flag的含义。？？？？？？
    for word,flag in words:
        if word not in stopwordslist and flag not in stop_flag:
            if word != '\t':
                res.append(word)
    return res

In [94]:
#将初步分词结果写入文件
def writeCorpus(fileName,contents):
    fp = open(fileName,'w+')
    fp.write(str(contents))
    fp.close()
    return True

In [95]:
#遍历初步分词的结果，进行非重复词的hash编码，2*32位应该足够，将idf作为权重进行运算，分别存入两个字典中
#设置所有词的hash值
def setHashAndWeight(path):
    f = codecs.open(path, 'r', encoding='utf-8')
    dictHash = dict()
    dictWeight = dict()
    i = 0 #hash编码
    lines = 0#记录文本数量,以计算idf值
    #遍历文本，进行hash编码和统计df词频(在多少篇文章出现过，而不是总词频，\
    #比如某个词在一个文本中出现三次也只算一次)
    for line in f:
        lines +=1
        text_noRepeate = set(str(line).strip().split())
        for item in text_noRepeate:
            if item not in dictWeight:
                dictWeight[item] = 1
                dictHash[item] = i
                i += 1
            else:
                dictWeight[item] += 1
    f.close()
    del i
    #hash编码转为array形式的二进制，方便计算
    for item in dictHash:
        L = list(bin(dictHash[item]))[2:]
        intL = [int(x) for x in L]
        for i in range(len(intL)):
            if intL[i] == 0:
                intL[i] = -1
        intL = (BITS - len(intL))*[-1]+intL
        dictHash[item] = np.array(intL)
   #根据词频计算idf值
    for item in dictWeight:
        dictWeight[item] = math.log(lines/dictWeight[item])
    
    return dictHash,dictWeight

In [96]:
#根据词的hash对句子进行hash编码
BITS = 31
def senHash(sen,dictHash,dictWeight):
    senHashCode = np.zeros(BITS)
    temp = set(str(sen).strip().split())
    #temp = str(sen).strip().split()
    for item in temp:
        senHashCode += dictHash[item]*dictWeight[item]
    for i in range(BITS):
        if senHashCode[i] > 0:
            senHashCode[i] = 1
        else:
            senHashCode[i] = 0
    return senHashCode

#获取两个句子的Hamming distance，dis越小说明相似度越高
def sen2senDis(sen1, sen2,dictHash,dictWeight):
    temp1 = senHash(sen1,dictHash,dictWeight)
    temp2 = senHash(sen2,dictHash,dictWeight)
    Hamming = 0
    for i in range(BITS):
        if temp1[i] != temp2[i]:
            Hamming += 1
    return Hamming

In [100]:
s1 = tokenization('/Users/taoxudong/huli/test/高血压.txt')
s2 = tokenization('/Users/taoxudong/huli/test/低血压.txt')
s1_content = writeCorpus('test1',s1)
s2_content = writeCorpus('test2',s2)
#设置所有词的hash值和权重
dictHash,dictWeight = setHashAndWeight('./text_corpus')
haming = sen2senDis(s1,s2,dictHash,dictWeight)

KeyError: "'发生率']"